In [10]:
import pandas as pd
import psycopg2
import requests
import json
import numpy as np

In [11]:
# Insert query
q_ct_business_insert = """
INSERT INTO ct_business (
    credentialid,
    name,
    type,
    fullcredentialcode,
    credentialtype,
    credentialnumber,
    credential,
    status,
    active,
    issuedate,
    effectivedate,
    expirationdate,
    address,
    city,
    state,
    zip,
    recordrefreshedon,
    statusreason,
    businessname,
    credentialsubcategory,
    dba
)

VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
ON CONFLICT (credentialid) DO NOTHING;
"""

In [12]:
# Get data
response = requests.get("https://data.ct.gov/resource/ngch-56tr.json")

In [13]:
response_json = response.json()
response_json

[{'credentialid': '10',
  'name': 'SEMYON RODKIN',
  'type': 'INDIVIDUAL',
  'fullcredentialcode': 'PEN.0018966',
  'credentialtype': 'PEN',
  'credentialnumber': '18966',
  'credential': 'PROFESSIONAL ENGINEER',
  'status': 'INACTIVE',
  'statusreason': 'FAILED TO RENEW',
  'active': '0',
  'issuedate': '1995-03-13T00:00:00.000',
  'effectivedate': '2007-02-01T00:00:00.000',
  'expirationdate': '2008-01-31T00:00:00.000',
  'address': '214 WEST 29TH ST',
  'city': 'NEW YORK',
  'state': 'NY',
  'zip': '10001',
  'recordrefreshedon': '2009-06-23T00:00:00.000'},
 {'credentialid': '100',
  'name': 'RAILROAD SALVAGE OF CT INC',
  'type': 'CORPORATION',
  'businessname': 'RAILROAD SALVAGE OF CT INC',
  'fullcredentialcode': 'PME.0003731',
  'credentialtype': 'PME',
  'credentialnumber': '3731',
  'credential': 'NON LEGEND DRUG PERMIT',
  'status': 'INACTIVE',
  'active': '0',
  'expirationdate': '1995-12-31T00:00:00.000',
  'address': '70 BRITANNIA ST',
  'city': 'MERIDEN',
  'state': 'CT',

In [ ]:
for listing in 

In [ ]:
first_biz = response_json[0]
first_biz

In [ ]:
businesses = []

for biz in response_json:
    business_values = []
    for key, value in biz.items():
        business_values.append(value)
    businesses.append(business_values)

In [ ]:
converted = []
for business in businesses:
    converted_business = [int(value) if isinstance(value, np.int64) else value for value in business]
    converted.append(converted_business)

In [ ]:
converted

In [ ]:
# Save data in .txt file in order to not hit the endpoint repeatedly
with open("data.txt", 'w') as file:
    json.dump(response_json, file)

In [ ]:
df = pd.read_json('data.txt')

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
single_business = list(df.loc[0,].values)

In [ ]:
# Print out the single business
single_business

In [ ]:
# Convert numpy integers to Postgres compatible integers
single_business_converted_int = [int(value) if isinstance(value, np.int64) else value for value in single_business]

In [ ]:
# Check data types by printing a tuple of each item in the list
[(item, type(item)) for item in single_business_converted_int]

## Insert one value into the database

In [ ]:
# Connect to database
conn = psycopg2.connect("host=127.0.0.1 dbname=ct_business_db user=postgres password=Summer2024@")
conn.set_session(autocommit=True)

# Create cursor
cur = conn.cursor()
cur.execute(q_ct_business_insert, single_business_converted_int)

In [ ]:
cur.execute("""SELECT * FROM ct_business""")
rows = cur.fetchall()
for row in rows:
    print(row)

In [ ]:
conn.close()